In [59]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import py_vollib_vectorized as vollib
from datetime import datetime, timedelta

TRADIER_API_KEY = "Jta3vCKbfk5HxsN8hHqEp4HlyPku"
POLYGON_API_KEY = "sZcrR6OUpxYlIcSF5BnpeJphWaXbhkpi"
headers = {'Authorization': "Bearer {}".format(TRADIER_API_KEY), 'Accept': 'application/json'}
BASE_URL = "https://sandbox.tradier.com"

In [65]:

def get_historical_data_for_security(ticker,start=str((datetime.today() - timedelta(days=365)).date()),end=str(datetime.today().date()),api_key = POLYGON_API_KEY,df=True):
    response = requests.get(f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{start}/{end}?unadjusted=false&sort=asc&apiKey={api_key}")
    data = response.json()['results']
    if df: 
        data = pd.DataFrame(data)
        data['t'] = pd.to_datetime(data['t'],unit='ms')
        return data
    return data
    
def get_first_val_in_table(table):
    first_row = table.find('tr')
    if first_row:
        first_field = first_row.find('td')
        if first_field:
            enclosed = first_field.find('div')
            if enclosed:
                return enclosed.text
    return ''

def get_si_data_from_table(table,df=True):
    data = []
    cols = []
    rows = table.findAll('tr')
    for row in rows:
        col,val = row.findAll('td')
        cols.append(col.find('div').text.strip())
        data.append(val.text.strip())
    if df: return pd.DataFrame([data[2:]],columns=cols[2:]) # first 2 fields are bs
    return cols,data
        

def get_short_interest_for_security(ticker):
    response = requests.get("https://shortsqueeze.com/?symbol={}&submit=Short+Quote%E2%84%A2".format(ticker))
    soup = BeautifulSoup(response.text,'html.parser')
    si_table = [table for table in soup.findAll('table') if get_first_val_in_table(table) == 'Short Squeeze Ranking™'][0]
    return get_si_data_from_table(si_table)


def get_options_for_security(stock):
    response = requests.get('{}/v1/markets/options/lookup'.format(BASE_URL),
        params={'underlying': stock},
        headers=headers
    )

    json_response = response.json()
    return json_response['symbols'][0]['options']


def get_quotes_for_options(options,batch_size=25):
    chains = []
    for i in range(0, len(options),batch_size):
        options_batch = options[i:i+batch_size]
        response = requests.get('{}/v1/markets/quotes'.format(BASE_URL),
            params={'symbols': ','.join(options_batch), 'greeks': 'true'},
            headers=headers
        )    
        json_response = response.json()
        chains += json_response['quotes']['quote']
    return chains
    
def get_quote_for_security(stock):
    response = requests.get('{}/v1/markets/quotes'.format(BASE_URL),
        params={'symbols': stock, 'greeks': 'false'},
        headers=headers
    )    
    json_response = response.json()
    return json_response['quotes']['quote']
    

def get_most_shorted_stocks(df=True):
    table_cols = ['Symbol\nSymbol',
                 'Company Name',
                 'Price',
                 'Chg% (1D)',
                 'Chg% (YTD)',
                 'Short Interest',
                 'Short Date',
                 'Float',
                 'Float Shorted (%)']
    url = "https://www.marketwatch.com/tools/screener/short-interest"
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    si_table = soup.findAll('table')[0]
    si_cols =  [col.text.strip('\n') for col in si_table.find('thead').find('tr').findAll('th')]
    assert(si_cols == table_cols)
    rows = si_table.findAll('tr')[1:]
    data = []
    for row in rows: data.append([val.text.strip('\n') for val in row.findAll('td')])
    if df: return pd.DataFrame(data,columns=si_cols)
    return data


In [66]:
get_historical_data_for_security('AAPL')


,v,vw,o,c,h,l,t,n
0,200145124.0,85.0132,86.1800,84.7000,86.9500,83.5558,2020-06-12 04:00:00,514308
1,138808720.0,84.8782,83.3125,85.7475,86.4200,83.1450,2020-06-15 04:00:00,368615
2,165428728.0,87.7181,87.8650,88.0200,88.3000,86.1800,2020-06-16 04:00:00,446656
3,114388504.0,88.4283,88.7875,87.8975,88.8500,87.7725,2020-06-17 04:00:00,321829
4,96820384.0,87.8912,87.8525,87.9325,88.3625,87.3050,2020-06-18 04:00:00,274412
...,...,...,...,...,...,...,...,...
247,71057550.0,125.3691,126.1700,125.9000,126.3200,124.8321,2021-06-07 04:00:00,504254
248,74403774.0,127.0742,126.6000,126.7400,128.4600,126.2101,2021-06-08 04:00:00,532078
249,56695219.0,127.0733,127.2100,127.1300,127.7500,126.5200,2021-06-09 04:00:00,382625
250,71186421.0,126.6607,127.0200,126.1100,128.1900,125.9400,2021-06-10 04:00:00,498371


'2020-06-12'